In [236]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
def mard(df, c1, c2):
    a = abs(df[c1]-df[c2])/(df[c1]+df[c2])
    return a.sum()/a.count()

# Loading Reference name to Taxa Id file

In [4]:
ref = "/mnt/scratch2/avi/meta-map/kraken/KrakenDB/seqid2taxid.map"
with open(ref) as f:
    refId2TaxId = pd.read_csv(f, header=None, sep="\t", names=["refName", "refId"])

In [420]:
refId2TaxId[refId2TaxId.refId == 964]

,refName,refId


In [416]:
refId2TaxId[refId2TaxId.refName == 'gi|330822653|ref|NC_015422.1']

,refName,refId


# Loading Taxonomy Tree
### Taxum ID , Taxum Rank , Taxum Parent ID

In [31]:
tf = "/mnt/scratch2/avi/meta-map/kraken/KrakenDB/taxonomy/nodes.dmp"
taxa = []
id2rank = {}
with open(tf) as f:
    for line in f:
        toks = line.rstrip("\t|\n").split("\t|\t")
        taxa += [[int(toks[0]), int(toks[1]), toks[2]]]
        id2rank[int(toks[0])] = toks[2]
taxa_df = pd.DataFrame.from_records(taxa)
taxa_df.columns = ["taxaId", "pid", "rank"]

In [32]:
taxa_df.head()

,taxaId,pid,rank
0,1,1,no rank
1,2,131567,superkingdom
2,6,335928,genus
3,7,6,species
4,9,32199,species


In [421]:
a = taxa_df[taxa_df.taxaId == 964]
a

,taxaId,pid,rank
742,964,963,species


# List of Leaves

Right join of **pid** and **cid** (parent and child)
And then looking for any child that has never been a parent : with parent = **nan**

In [74]:
leaves = pd.merge(taxa_df[['pid']], taxa_df[['taxaId', 'rank']], left_on='pid', right_on='taxaId', how='right')

In [75]:
leaves = leaves[np.isnan(leaves['pid'])][['taxaId', 'rank']]
print(len(leaves))
leaves.head()

1528122


,taxaId,rank
1669515,25,species
1669516,27,species
1669517,28,species
1669518,38,species
1669519,45,species


# List of Roots
Left join of **pid** and **cid** (parent and child) And then looking for any parent that is never a child : child = **nan** parent

In [81]:
roots = pd.merge(taxa_df[['pid', 'rank']], taxa_df[['taxaId', 'pid']], left_on='pid', right_on='taxaId', how='left')
#roots = roots[np.isnan(roots['taxaId'])][['pid', 'rank']]
roots[roots['taxaId'] == roots['pid_y']]

,pid_x,rank,taxaId,pid_y
0,1,no rank,1,1
8284,1,superkingdom,1,1
9909,1,superkingdom,1,1
9931,1,no rank,1,1
12110,1,no rank,1,1
101269,1,no rank,1,1


In [78]:
c2p = {}
c2pid = {}
with open(tf) as f:
    for line in f:
        toks = line.rstrip("\t|\n").split("\t|\t")
        c2pid[int(toks[0])] = int(toks[1])
        if id2rank[int(toks[0])] not in c2p:
            c2p[id2rank[int(toks[0])]] = set()
        c2p[id2rank[int(toks[0])]].add(id2rank[int(toks[1])])

# Child -> Parent

In [24]:
for key, value in c2p.items():
    print ('{} --> parent list:'.format(key))
    print(value)
    print('\n')

no rank --> parent list:
{'genus', 'no rank', 'superorder', 'species subgroup', 'superclass', 'infraorder', 'superfamily', 'tribe', 'superkingdom', 'parvorder', 'subtribe', 'suborder', 'cohort', 'subspecies', 'species', 'phylum', 'infraclass', 'order', 'varietas', 'subphylum', 'subfamily', 'subclass', 'family', 'subkingdom', 'class', 'forma', 'species group', 'subgenus', 'kingdom'}


superkingdom --> parent list:
{'no rank'}


genus --> parent list:
{'no rank', 'suborder', 'family', 'class', 'phylum', 'infraclass', 'order', 'superkingdom', 'subtribe', 'superfamily', 'tribe', 'subfamily', 'subphylum', 'subclass'}


species --> parent list:
{'genus', 'no rank', 'suborder', 'family', 'class', 'species', 'phylum', 'species subgroup', 'species group', 'order', 'subgenus', 'superfamily', 'tribe', 'subfamily', 'subclass'}


order --> parent list:
{'no rank', 'superorder', 'cohort', 'class', 'phylum', 'infraclass', 'subphylum', 'superkingdom', 'subclass'}


family --> parent list:
{'no rank', 

# ranks

In [12]:
print(len(taxa_df['rank'].unique()))
taxa_df['rank'].unique()

30


array(['no rank', 'superkingdom', 'genus', 'species', 'order', 'family',
       'subspecies', 'subfamily', 'tribe', 'phylum', 'class', 'forma',
       'suborder', 'subclass', 'varietas', 'kingdom', 'subphylum',
       'superfamily', 'infraorder', 'infraclass', 'superorder', 'subgenus',
       'superclass', 'parvorder', 'superphylum', 'species group',
       'species subgroup', 'cohort', 'subtribe', 'subkingdom'], dtype=object)

# Both Parent and Child at the same time in the reference list? 
## yes!

In [87]:
ref2taxaLevel = pd.merge(refId2TaxId, taxa_df, left_on="refId", right_on="taxaId")
print(len(refId2TaxId))
print(len(ref2taxaLevel))
ref2taxaLevel.head()

4815
4815


,refName,refId,taxaId,pid,rank
0,gi|9791176|ref|NC_002180.1|,1986029,1986029,40274,species
1,gi|10803547|ref|NC_001869.1|,64091,64091,2242,no rank
2,gi|15789340|ref|NC_002607.1|,64091,64091,2242,no rank
3,gi|16119979|ref|NC_002608.1|,64091,64091,2242,no rank
4,gi|10954488|ref|NC_001732.1|,243232,243232,2190,no rank


In [149]:
ref2taxaLevel[ref2taxaLevel['taxaId'] == 680]

,refName,refId,taxaId,pid,rank


In [27]:
a = set()
for t in ref2taxaLevel.taxaId:
    a.add(t)

In [29]:
b = set()
for t in ref2taxaLevel.taxaId:
    b.add(c2pid[t])

List of reference taxa that are parents of some children in the same list

In [30]:
a.intersection(b)

{316,
 340,
 813,
 1148,
 1604,
 1639,
 2110,
 29447,
 40041,
 91891,
 107806,
 190650,
 192222,
 196627,
 208964,
 523796,
 543891,
 869727,
 907287,
 941967,
 1001534,
 1071763,
 1263406,
 1283330,
 1306414}

# Reference Rank Distribution
### Note: Not all the "no rank"s are the same!!

In [42]:
ref2taxaLevel.groupby('rank').count()

,refName,refId,taxaId
rank,,,
no rank,4336,4336,4336
species,465,465,465
subspecies,14,14,14


# Analysis of the truth 

In [100]:
truthfile = '/mnt/scratch2/avi/meta-map/kraken/meta/truth.txt'
truth = pd.read_csv(truthfile, sep="\t")

In [101]:
truth.head()

,taxid,counts,species,size,dataset
0,79329,24666,Chitinophaga pinensis,9127347,Huttenhower_HC1
1,1836,22195,Saccharopolyspora erythraea,8079083,Huttenhower_HC1
2,152480,20654,Burkholderia ambifaria,8095900,Huttenhower_HC1
3,182640,20483,Kribbella flavida,7579488,Huttenhower_HC1
4,80866,18289,Delftia acidovorans,6823175,Huttenhower_HC1


In [102]:
truth = pd.merge(truth, taxa_df, left_on='taxid', right_on='taxaId')
truth.head()

,taxid,counts,species,size,dataset,taxaId,pid,rank
0,79329,24666,Chitinophaga pinensis,9127347,Huttenhower_HC1,79329,79328,species
1,79329,3127,Chitinophaga pinensis,9127347,Huttenhower_LC1,79329,79328,species
2,1836,22195,Saccharopolyspora erythraea,8079083,Huttenhower_HC1,1836,1835,species
3,1836,125,Saccharopolyspora erythraea,8079083,Huttenhower_LC1,1836,1835,species
4,152480,20654,Burkholderia ambifaria,8095900,Huttenhower_HC1,152480,87882,species


### Everything has been simulated from "species" level

In [105]:
truth['rank'].unique()

array(['species'], dtype=object)

In [429]:
for dataset in ['HC1', 'HC2', 'LC1', 'LC2', 'LC3', 'LC4', 'LC5', 'LC6', 'LC7', 'LC8']:
    print('\n{}'.format(dataset))
    hc1 = truth[truth.dataset == 'Huttenhower_'+dataset]
    #print(hc1['counts'].sum())
    krakmap_out = '~/projects/pufferfish/build/'+dataset+'_species.out'
    avi_krakmap_file = '/mnt/scratch2/avi/meta-map/kraken/puff/fat_out/'+dataset+'_report.txt'
    krakmap = pd.read_csv(krakmap_out, sep="\t")
    #print(krakmap['count'].sum())
    krakmap = krakmap[krakmap['taxaId'] != 0]
    #print(krakmap['count'].sum())
    #print('fatal original count : {}'.format(len(krakmap['count'])))
    avi_krakmap = pd.read_csv(avi_krakmap_file, sep='\t')
    #print(avi_krakmap['Count'].sum())
    avi_krakmap = avi_krakmap[avi_krakmap['Feature'] != 0]
    #print(avi_krakmap['Count'].sum())
    #avi_krakmap = avi_krakmap[avi_krakmap['Feature'] != 0]
    #print('avi original count: {}'.format(len(avi_krakmap)))
    avi_krakmap = pd.merge(avi_krakmap, taxa_df, left_on='Feature', right_on='taxaId')
    #print('count after merging to get ranks (should be the same): {}'.format(len(avi_krakmap)))
    fatal_species = krakmap[krakmap.taxaRank == 'species']
    avi_species = avi_krakmap[avi_krakmap['rank'] == 'species']
    #print('fatal count of species : {}'.format(len(krakmap_species)))
    #print('avi count of species : {}'.format(len(avi_krakmap)))
    pythcppjoined = pd.merge(krakmap, avi_krakmap, left_on='taxaId', right_on='Feature', how='outer').fillna(0)
    print('Fatal vs Avi')
    print(spearmanr(pythcppjoined['count'], pythcppjoined['Count']))
    print('mard: {}'.format(mard(pythcppjoined, 'count', 'Count')))
    pythcppjoined['diff'] = pythcppjoined['count']-pythcppjoined['Count']
    print(pythcppjoined[pythcppjoined['diff'] != 0][['taxaId_x', 'Feature', 'taxaRank', 'rank', 'Count', 'diff']])
    #print('avi # of reads: {}'.format(avi_krakmap['Count'].sum()))
    #print('fatal # of reads: {}'.format(krakmap_species['count'].sum()))
    kraktruthjoined = pd.merge(hc1, fatal_species, left_on='taxid', right_on='taxaId', how='outer').fillna(0)
    print('FatalTruth')
    print(kraktruthjoined[['counts', 'count']].corr(method='spearman'))
    print('mard: {}'.format(mard(kraktruthjoined, 'counts', 'count')))
    #print('diff:')
    #kraktruthjoined['diff'] = kraktruthjoined['counts']-kraktruthjoined['count']
    #print(kraktruthjoined[['taxid','diff']])
    pythtruthjoined = pd.merge(hc1, avi_species, left_on='taxid', right_on='Feature', how='outer').fillna(0)
    print('AviTruth')
    print(pythtruthjoined[['counts', 'Count']].corr(method='spearman'))
    print('mard: {}'.format(mard(pythtruthjoined, 'counts', 'Count')))
    #print(pythtruthjoined['counts']-pythtruthjoined['Count'])


HC1
Fatal vs Avi
SpearmanrResult(correlation=1.0, pvalue=0.0)
mard: 9.852656441893663e-07
     taxaId_x  Feature       taxaRank           rank  Count  diff
199       339      339        species        species  10964     1
208      1396     1396        species        species   6268     1
239    111527   111527  species group  species group   5873    -1
260     28450    28450        species        species   2969     1
267    182640   182640        species        species  16667     1
FatalTruth
          counts     count
counts  1.000000  0.836454
count   0.836454  1.000000
mard: 0.6913114348184081
AviTruth
          counts     Count
counts  1.000000  0.836454
Count   0.836454  1.000000
mard: 0.6913123818230174

HC2
Fatal vs Avi
SpearmanrResult(correlation=1.0, pvalue=0.0)
mard: 8.109578653934357e-07
     taxaId_x  Feature taxaRank     rank  Count  diff
15        645      645  species  species  11602     1
164       543      543   family   family   7070    -1
234      1718     1718  spec